# XDAC Inference Example

250616


In [ ]:
## Install
!pip install captum

In [1]:
## 0) download XDAC model/data from huggingface (90s)
import os

XDAC_root_path = './XDAC_obs'

from huggingface_hub import snapshot_download
snapshot_download(
    repo_id="keepsteady/XDAC_obs",
    local_dir=XDAC_root_path,
    local_dir_use_symlinks=False
)

C:\Users\user\anaconda3\envs\venv\Lib\site-packages\huggingface_hub\file_download.py:834: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

README.md: 0.00B [00:00, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

XDAC_D.py: 0.00B [00:00, ?B/s]

XDAC_Unified.py: 0.00B [00:00, ?B/s]

XDAC_A.py: 0.00B [00:00, ?B/s]

'C:\\Users\\user\\Desktop\\250505_acl논문정리\\250616_공개모델정리\\XDAC_obs'

In [2]:
## 1) Load Korean LGC dataset
import json
from datasets import Dataset
from pprint import pprint

path_data = os.path.join(XDAC_root_path, './LGC_data/LGC_data_v1.0.json')

with open(path_data, 'r', encoding='utf-8') as f:
    data_list = json.load(f)
    dataset_LGC = Dataset.from_list(data_list)

print(dataset_LGC)
pprint(dataset_LGC[-1])

Dataset({
    features: ['idx', 'comment_language', 'llm_model_family', 'llm_model_selection', 'temperature', 'enhancing_comment_naturalness', 'sentiment', 'sentiment_subtype_selection', 'reference_augmented_generation_strategy', 'reference_comment_input', 'reference_news_input', 'reference_opinion_input', 'generated_comment', 'toxicity'],
    num_rows: 2000
})
{'comment_language': 'Korean',
 'enhancing_comment_naturalness': '',
 'generated_comment': '이런 망나니가 국회의원이라니 XXXX 정치는 끝났다. 국민들이 다 바보인가?',
 'idx': 6396,
 'llm_model_family': 'Claude',
 'llm_model_selection': 'claude-3-5-haiku-20241022',
 'reference_augmented_generation_strategy': 'Opinion-based Generation',
 'reference_comment_input': True,
 'reference_news_input': True,
 'reference_opinion_input': True,
 'sentiment': 'negative',
 'sentiment_subtype_selection': 'Promotion of Social Division: '
                                'Discrimination/Blame of Specific Groups',
 'temperature': 0.9,
 'toxicity': True}


In [3]:
## 2) Load XDAC
from XDAC_obs.XDAC_protected import AIUnifiedEngine

print("XDAC Unified Engine: AI Detection & Attribution")
print("=" * 60)

device = 'cuda'
# device = 'cpu'

# Initialize unified engine
unified_engine = AIUnifiedEngine(
    detection_model_path=os.path.join(XDAC_root_path, 'XDAC-D'),    # Path to XDAC-D model
    attribution_model_path=os.path.join(XDAC_root_path, 'XDAC-A'),  # Path to XDAC-A model
    device=device,                      # or 'cpu', or None for auto-detection
    xai_enabled=True                    # Enable XAI analysis
)

XDAC Unified Engine: AI Detection & Attribution
XDAC Unified Engine: Detection & Attribution
Initializing Detection Engine (XDAC-D)...
Loading XDAC-D model from: ./XDAC_obs\XDAC-D
Using device: cuda
Model loaded successfully!

Initializing Attribution Engine (XDAC-A)...
Loading XDAC-A model from: ./XDAC_obs\XDAC-A
Using device: cuda
Model loaded successfully!

Unified Engine initialized successfully!
Device: cuda
XAI enabled: True


In [4]:
## 3) Run XDAC
multiple_texts = [
    '사이비 종교 뿌리 뽑자',
    '전기차가 친환경이라는 것은 기업들의 이익을 위한 합리화일 뿐이다.',
    '숄츠 총리가 한국에 온 것은 좋지만, 북한 문제를 언급하는 것은 한국 내 정치적 갈등을 유발할 수 있지 않을까요?',
    '이 기사 정말 웃기네요 ㅋㅋㅋㅋㅋ',
    '정부 정책에 대해 더 많은 토론이 필요합니다.',
    '오늘 날씨가 참 좋네요! 산책하기 딱 좋은 날씨입니다.',
    'AI 기술의 발전으로 우리 생활이 많이 편해졌습니다.'
]

# Unified inference
results = unified_engine.predict(multiple_texts, batch_size=10)
unified_engine.print_results(results, save_path='result_XDAC.txt')
unified_engine.save_xai_results_to_html(results, html_file_path='result_XDAC.html')

# Get top predictions for detailed analysis
top_results = unified_engine.get_top_predictions(results, top_k=3)

=== Stage 1: AI Detection (1 samples) ===


XAI predicting: 100%|████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.25it/s]

=== Stage 2: AI Attribution (0 LLM samples) ===

AI Unified Detection & Attribution Results (1 samples) 1.2/s | 0.81s
  1. 사이비 종교 뿌리 뽑자                                       -> Human ( 98.2%)
  => XAI Important: ['종교']


--------------------------------------------------------------------------------

Detection Summary: Human: 1, LLM: 0

✅ Results saved to: result_XDAC.txt
XAI results saved to: result_XDAC.html
